# Repuestos

In [1]:
##C:\Users\U1205486\AppData\Roaming\SAP\SAP GUI\Scripts
## "C:\Program Files (x86)\SAP\FrontEnd\SAPgui\saplogon.exe"

##SCRIPT TEST 

import win32com.client
import sys
import pandas as pd
import datetime
import subprocess
import time
import openpyxl
import pyperclip
import os

from config.helper import conectar_usuario, close_excel_file
from config.fechas import obtener_primer_y_ultimo_dia_mes, obtener_primer_y_ultimo_dia_semana, diferencia_en_dias
from config.Login_sap import *

import os
import warnings

warnings.filterwarnings("ignore")
pd.options.display.max_columns = None

## Proceso

Debemos hacer merge entre dos archivos provenientes de SAP, KOB1 y MB25, la cual contiene la información a solicitar vs la información cargada en sistema

In [2]:
# fecha = '13-05-2024'
# print(fecha)

In [3]:
fecha = datetime.date.today().strftime("%d-%m-%Y")
print(fecha)


try:
  kob = pd.read_excel(ruta_sap_documentos + "/KOB OS {}.XLSX".format(fecha))

except:
  kob = pd.read_excel(ruta_sap_documentos + "/KOB OS {}.xlsx".format(fecha))
print("Longitud KOB:",len(kob))

try:
  mb = pd.read_excel(ruta_sap_documentos + "/MB25 OS {}.XLSX".format(fecha))

except:
  mb = pd.read_excel(ruta_sap_documentos + "/MB25 OS {}.xlsx".format(fecha))
print("Longitud MB:",len(mb))

### KOB1 ###

### Convertir la columna a enteros manteniendo los valores vacíos
kob['Orden'] = pd.to_numeric(kob['Orden'], errors='coerce').astype('Int64')
kob['Material'].fillna('', inplace=True)

### Creamos la llave
kob['KEY'] = kob['Material'] + kob['Orden'].astype(str)

29-08-2024
Longitud KOB: 1289
Longitud MB: 1790


In [4]:
mb.columns

Index(['Material', 'Texto breve de material', 'Cantidad necesaria',
       'Cantid.reducidas', 'Cantidad diferencia', 'Nombre del usuario',
       'Fecha de necesidad', 'Orden', 'Nº reserva', 'Centro'],
      dtype='object')

In [5]:
### Crear la tabla dinámica KOB01
pivot_df_kob = pd.pivot_table(kob, index=['KEY'], values='Cantidad total reg.', aggfunc="sum")
pivot_df_kob_ordenado = pivot_df_kob.reset_index()

pivot_df_kob_ordenado.rename(columns={'Cantidad total reg.':'Cantid.reducidas' }, inplace = True)


### MB25 ###

### Formateamos la fecha a dd-mm-yyyy
mb['Fecha de necesidad'] = mb['Fecha de necesidad'].dt.strftime("%d-%m-%Y")

### Transformamos las variables a Texto
mb['Centro'] = mb['Centro'].astype(str)

### Creamos la llave
mb['KEY'] = mb['Material'] + mb['Orden'].astype(str)

### Crear la tabla dinámica MB25
pivot_df = pd.pivot_table(mb, index=['KEY'], values='Cantidad necesaria', aggfunc="sum")
pivot_df_ordenado = pivot_df.reset_index()

### Eliminamos los duplicados
mb_unicos = mb.drop_duplicates(subset=['KEY'], keep='first')

### Renombramos las columnas a eliminar
mb_unicos.rename(columns={'Cantidad necesaria':'borrar1',
                          'Cantid.reducidas':'borrar2',
                          'Cantidad diferencia':'borrar3'


}, inplace = True)


### Hacer merge del DataFrame original con la tabla pivot
df_mb = pd.merge(mb_unicos, pivot_df_ordenado, on='KEY')

### Generamos el formato a donde debería pegarse la información

In [6]:
### Junte tablas
df = pd.merge(df_mb, pivot_df_kob_ordenado, on='KEY', how='left')

### Reemplazamos los valores nulos con 0
df['Cantid.reducidas'].fillna(0, inplace=True)

### Calculamos la diferencia
df['Cantidad diferencia'] = df['Cantidad necesaria'] - df['Cantid.reducidas']

df['Orden'] = df['Orden'].astype(str)
df['Nº reserva'] = df['Nº reserva'].astype(str)

df["espacio1"] = ""
df["espacio2"] = ""
df["espacio3"] = ""
df["espacio4"] = ""
df["espacio5"] = ""


### Seleccionamos las columnas a utilizar
df = df[['KEY','Material', 'Texto breve de material', 'Cantidad necesaria',
       'Cantid.reducidas', 'Cantidad diferencia', 'Nombre del usuario',
       'Fecha de necesidad', 'Orden','espacio1','espacio2','espacio3',
         'espacio4','espacio5','Nº reserva', 'Centro']]



df.to_excel(f"Respuestos {fecha}.xlsx", index = False)

### TEST KOB

In [ ]:
kob.dtypes

Orden                             float64
Clase de coste                    float64
Material                           object
Texto breve de material            object
Cantidad total reg.               float64
Valor/Moneda objeto               float64
Val/Mon.so.CO                     float64
Operación                          object
Fe.contabilización         datetime64[ns]
Denom.clase de coste               object
Documento compras                 float64
Denominación del objeto            object
Clase de objeto                    object
Clase de objeto.1                  object
Denominación                       object
Objeto                            float64
dtype: object

In [ ]:
# Convertir la columna a enteros manteniendo los valores vacíos
kob['Orden'] = pd.to_numeric(kob['Orden'], errors='coerce').astype('Int64')

kob['Material'].fillna('', inplace=True)

In [ ]:
kob.head(3)

,Orden,Clase de coste,Material,Texto breve de material,Cantidad total reg.,Valor/Moneda objeto,Val/Mon.so.CO,Operación,Fe.contabilización,Denom.clase de coste,Documento compras,Denominación del objeto,Clase de objeto,Clase de objeto.1,Denominación,Objeto
0,400364537,9100410.0,NaN,NaN,8.0,200.0,200.0,RKL,2024-01-25,Costo HH Normal,NaN,IMPLEMENTACION DE EQUIPO FULL,Orden,ORD,IMPLEMENTACION DE EQUIPO,400364537.0
1,400364537,9100410.0,NaN,NaN,8.0,200.0,200.0,RKL,2024-01-25,Costo HH Normal,NaN,IMPLEMENTACION DE EQUIPO FULL,Orden,ORD,IMPLEMENTACION DE EQUIPO,400364537.0
2,400364963,9100410.0,NaN,NaN,8.0,200.0,200.0,RKL,2024-02-05,Costo HH Normal,NaN,EVALUACION POR TERMINO DE CONTRATO,Orden,ORD,EVALUACION DE EQUIPO,400364963.0


In [ ]:
kob.columns

Index(['Orden', 'Clase de coste', 'Material', 'Texto breve de material',
       'Cantidad total reg.', 'Valor/Moneda objeto', 'Val/Mon.so.CO',
       'Operación', 'Fe.contabilización', 'Denom.clase de coste',
       'Documento compras', 'Denominación del objeto', 'Clase de objeto',
       'Clase de objeto.1', 'Denominación', 'Objeto'],
      dtype='object')

In [ ]:
kob['Material'].fillna('', inplace=True)

In [ ]:
kob['KEY'] = kob['Material'] + kob['Orden'].astype(str)

In [ ]:
### Formateamos la fecha a dd-mm-yyyy
kob['Fe.contabilización'] = kob['Fe.contabilización'].dt.strftime("%d-%m-%Y")

In [ ]:
kob.head(3)

,Orden,Clase de coste,Material,Texto breve de material,Cantidad total reg.,Valor/Moneda objeto,Val/Mon.so.CO,Operación,Fe.contabilización,Denom.clase de coste,Documento compras,Denominación del objeto,Clase de objeto,Clase de objeto.1,Denominación,Objeto,KEY
0,400364537,9100410.0,,NaN,8.0,200.0,200.0,RKL,25-01-2024,Costo HH Normal,NaN,IMPLEMENTACION DE EQUIPO FULL,Orden,ORD,IMPLEMENTACION DE EQUIPO,400364537.0,400364537
1,400364537,9100410.0,,NaN,8.0,200.0,200.0,RKL,25-01-2024,Costo HH Normal,NaN,IMPLEMENTACION DE EQUIPO FULL,Orden,ORD,IMPLEMENTACION DE EQUIPO,400364537.0,400364537
2,400364963,9100410.0,,NaN,8.0,200.0,200.0,RKL,05-02-2024,Costo HH Normal,NaN,EVALUACION POR TERMINO DE CONTRATO,Orden,ORD,EVALUACION DE EQUIPO,400364963.0,400364963


In [ ]:
### TEST ###
test = kob[kob['Material']!=""]
test[test['KEY'].duplicated()].head(3)

,Orden,Clase de coste,Material,Texto breve de material,Cantidad total reg.,Valor/Moneda objeto,Val/Mon.so.CO,Operación,Fe.contabilización,Denom.clase de coste,Documento compras,Denominación del objeto,Clase de objeto,Clase de objeto.1,Denominación,Objeto,KEY
318,400366206,41013104.0,6134-11-5120,ESPACIADORES,1.0,5.44,5.44,COIN,2024-02-22,Costo Vta Repto Serv,NaN,reparacion marzo 2024,Orden,ORD,NaN,400366206.0,6134-11-5120400366206
341,400366544,41013104.0,FOCO-FAE-BLAN-REDO,"FOCO FAENERO LED BLANCO REDONDO 4""",2.0,17.54,17.54,COIN,2024-04-11,Costo Vta Repto Serv,NaN,Plan de Reparación Febrero 2023,Orden,ORD,NaN,400366544.0,FOCO-FAE-BLAN-REDO400366544
363,400367087,41013104.0,426-07-32442,FILTRO,1.0,27.80,27.80,COIN,2024-03-05,Costo Vta Repto Serv,NaN,Plan de Reparación Marzo 2024,Orden,ORD,NaN,400367087.0,426-07-32442400367087


In [ ]:
kob[kob['KEY'].duplicated()].head(1)

,Orden,Clase de coste,Material,Texto breve de material,Cantidad total reg.,Valor/Moneda objeto,Val/Mon.so.CO,Operación,Fe.contabilización,Denom.clase de coste,Documento compras,Denominación del objeto,Clase de objeto,Clase de objeto.1,Denominación,Objeto,KEY
1,400364537,9100410.0,,NaN,8.0,200.0,200.0,RKL,2024-01-25,Costo HH Normal,NaN,IMPLEMENTACION DE EQUIPO FULL,Orden,ORD,IMPLEMENTACION DE EQUIPO,400364537.0,400364537


In [ ]:
### Crear la tabla dinámica
pivot_df_kob = pd.pivot_table(kob, index=['KEY'], values='Cantidad total reg.', aggfunc="sum")
pivot_df_kob_ordenado = pivot_df_kob.reset_index()

pivot_df_kob_ordenado.rename(columns={'Cantidad total reg.':'Cantid.reducidas'

}, inplace = True)

pivot_df_kob_ordenado

KeyError: 'Cantidad total reg.'

In [ ]:
kob[kob['KEY']=='6134-11-5120400366206']

,Orden,Clase de coste,Material,Texto breve de material,Cantidad total reg.,Valor/Moneda objeto,Val/Mon.so.CO,Operación,Fe.contabilización,Denom.clase de coste,Documento compras,Denominación del objeto,Clase de objeto,Clase de objeto.1,Denominación,Objeto,KEY
315,400366206,41013104.0,6134-11-5120,ESPACIADORES,11.0,58.56,58.56,COIN,2024-03-05,Costo Vta Repto Serv,NaN,reparacion marzo 2024,Orden,ORD,NaN,400366206.0,6134-11-5120400366206
318,400366206,41013104.0,6134-11-5120,ESPACIADORES,1.0,5.44,5.44,COIN,2024-02-22,Costo Vta Repto Serv,NaN,reparacion marzo 2024,Orden,ORD,NaN,400366206.0,6134-11-5120400366206


In [ ]:
pivot_df_kob_ordenado[pivot_df_kob_ordenado['KEY']=='6134-11-5120400366206']

,KEY,Cantidad total reg.
681,6134-11-5120400366206,12.0


In [ ]:
#kob.to_excel("Prueba_13.xlsx", index = False)

### MB

In [ ]:
mb.head(2)

,Material,Texto breve de material,Cantidad necesaria,Cantid.reducidas,Cantidad diferencia,Nombre del usuario,Fecha de necesidad,Orden,Nº reserva,Centro,KEY
0,100315-122,A.W. HYDRAULIC PLUS ISO 46 BL 19 LT,8,0,8,U1205222,2024-04-01,400361280,2638118,3036,100315-122400361280
1,FA-300PCS,FUELLE CILINDRO STICK PC300,1,0,1,U1205222,2024-04-01,400361280,2638118,3036,FA-300PCS400361280


In [ ]:
mb.columns

Index(['Material', 'Texto breve de material', 'Cantidad necesaria',
       'Cantid.reducidas', 'Cantidad diferencia', 'Nombre del usuario',
       'Fecha de necesidad', 'Orden', 'Nº reserva', 'Centro', 'KEY'],
      dtype='object')

In [ ]:
### Formateamos la fecha a dd-mm-yyyy
mb['Fecha de necesidad'] = mb['Fecha de necesidad'].dt.strftime("%d-%m-%Y")

In [ ]:
mb['Centro'] = mb['Centro'].astype(str)

In [ ]:
mb['KEY'] = mb['Material'] + mb['Orden'].astype(str)

In [ ]:
mb.head(2)

,Material,Texto breve de material,Cantidad necesaria,Cantid.reducidas,Cantidad diferencia,Nombre del usuario,Fecha de necesidad,Orden,Nº reserva,Centro,KEY
0,100315-122,A.W. HYDRAULIC PLUS ISO 46 BL 19 LT,8,0,8,U1205222,01-04-2024,400361280,2638118,3036,100315-122400361280
1,FA-300PCS,FUELLE CILINDRO STICK PC300,1,0,1,U1205222,01-04-2024,400361280,2638118,3036,FA-300PCS400361280


In [ ]:
mb[mb['KEY'].duplicated()].head(3)

,Material,Texto breve de material,Cantidad necesaria,Cantid.reducidas,Cantidad diferencia,Nombre del usuario,Fecha de necesidad,Orden,Nº reserva,Centro,KEY
38,01010-81230,TORNILLO,3,0,3,U1205222,2024-04-01,400361280,2638118,3036,01010-81230400361280
42,207-01-75190,MANGUERA,1,0,1,U1205222,2024-04-01,400361280,2638118,3036,207-01-75190400361280
44,20Y-62-22790,BANDA,1,0,1,U1205222,2024-04-01,400361280,2638118,3036,20Y-62-22790400361280


In [ ]:
mb[mb['KEY']=='01010-81230400361280']

,Material,Texto breve de material,Cantidad necesaria,Cantid.reducidas,Cantidad diferencia,Nombre del usuario,Fecha de necesidad,Orden,Nº reserva,Centro,KEY
35,01010-81230,TORNILLO,3,0,3,U1205222,2024-04-01,400361280,2638118,3036,01010-81230400361280
38,01010-81230,TORNILLO,3,0,3,U1205222,2024-04-01,400361280,2638118,3036,01010-81230400361280


In [ ]:
### Crear la tabla dinámica MB25
pivot_df = pd.pivot_table(mb, index=['KEY'], values='Cantidad necesaria', aggfunc="sum")
pivot_df_ordenado = pivot_df.reset_index()
pivot_df_ordenado

,KEY,Cantidad necesaria
0,00557617-P400364384,2
1,01010-61440400369366,3
2,01010-61455400367580,0
3,01010-61635400369306,1
4,01010-61640400367128,4
...,...,...
2826,UL228K4L42-P400368775,2
2827,US228K027-P400368775,2
2828,WA101788400370655,1
2829,WA130753400370655,2


In [ ]:
mb_unicos = mb.drop_duplicates(subset=['KEY'], keep='first')
mb_unicos.head(3)

,Material,Texto breve de material,Cantidad necesaria,Cantid.reducidas,Cantidad diferencia,Nombre del usuario,Fecha de necesidad,Orden,Nº reserva,Centro,KEY
0,100315-122,A.W. HYDRAULIC PLUS ISO 46 BL 19 LT,8,0,8,U1205222,2024-04-01,400361280,2638118,3036,100315-122400361280
1,FA-300PCS,FUELLE CILINDRO STICK PC300,1,0,1,U1205222,2024-04-01,400361280,2638118,3036,FA-300PCS400361280
2,FA-300PCV,FUELLE CILINDRO VOLTEO PC300,1,0,1,U1205222,2024-04-01,400361280,2638118,3036,FA-300PCV400361280


In [ ]:
mb_unicos.rename(columns={'Cantidad necesaria':'borrar1',
                          'Cantid.reducidas':'borrar2',
                          'Cantidad diferencia':'borrar3'


}, inplace = True)

In [ ]:
mb_unicos[mb_unicos['KEY']=='01010-81230400361280']

,Material,Texto breve de material,borrar,borrar2,borrar3,Nombre del usuario,Fecha de necesidad,Orden,Nº reserva,Centro,KEY
35,01010-81230,TORNILLO,3,0,3,U1205222,2024-04-01,400361280,2638118,3036,01010-81230400361280


In [ ]:
mb_unicos.columns

Index(['Material', 'Texto breve de material', 'Cantidad necesaria',
       'Cantid.reducidas', 'Cantidad diferencia', 'Nombre del usuario',
       'Fecha de necesidad', 'Orden', 'Nº reserva', 'Centro', 'KEY'],
      dtype='object')

In [ ]:
### Hacer merge del DataFrame original con la tabla pivot
df_mb = pd.merge(mb_unicos, pivot_df_ordenado, on='KEY')


### Junte con MB con KOB

In [ ]:
df = pd.merge(df_mb, pivot_df_kob_ordenado, on='KEY', how='left')
df.head(3)

,Material,Texto breve de material,borrar,borrar2,borrar3,Nombre del usuario,Fecha de necesidad,Orden,Nº reserva,Centro,KEY,Cantidad necesaria,Cantid.reducidas
0,100315-122,A.W. HYDRAULIC PLUS ISO 46 BL 19 LT,8,0,8,U1205222,2024-04-01,400361280,2638118,3036,100315-122400361280,8,8.0
1,FA-300PCS,FUELLE CILINDRO STICK PC300,1,0,1,U1205222,2024-04-01,400361280,2638118,3036,FA-300PCS400361280,1,NaN
2,FA-300PCV,FUELLE CILINDRO VOLTEO PC300,1,0,1,U1205222,2024-04-01,400361280,2638118,3036,FA-300PCV400361280,1,NaN


In [ ]:
df['Cantid.reducidas'].fillna(0, inplace=True)
df['Cantidad diferencia'] = df['Cantidad necesaria'] - df['Cantid.reducidas']
df.head(2)

,Material,Texto breve de material,borrar,borrar2,borrar3,Nombre del usuario,Fecha de necesidad,Orden,Nº reserva,Centro,KEY,Cantidad necesaria,Cantid.reducidas,Cantidad diferencia
0,100315-122,A.W. HYDRAULIC PLUS ISO 46 BL 19 LT,8,0,8,U1205222,2024-04-01,400361280,2638118,3036,100315-122400361280,8,8.0,0.0
1,FA-300PCS,FUELLE CILINDRO STICK PC300,1,0,1,U1205222,2024-04-01,400361280,2638118,3036,FA-300PCS400361280,1,0.0,1.0


In [ ]:
df = df[['KEY','Material', 'Texto breve de material', 'Cantidad necesaria',
       'Cantid.reducidas', 'Cantidad diferencia', 'Nombre del usuario',
       'Fecha de necesidad', 'Orden', 'Nº reserva', 'Centro']]
df.head(2)

,KEY,Material,Texto breve de material,Cantidad necesaria,Cantid.reducidas,Cantidad diferencia,Nombre del usuario,Fecha de necesidad,Orden,Nº reserva,Centro
0,100315-122400361280,100315-122,A.W. HYDRAULIC PLUS ISO 46 BL 19 LT,8,8.0,0.0,U1205222,2024-04-01,400361280,2638118,3036
1,FA-300PCS400361280,FA-300PCS,FUELLE CILINDRO STICK PC300,1,0.0,1.0,U1205222,2024-04-01,400361280,2638118,3036


In [ ]:
df[df['KEY']=='01010-81230400361280']

,KEY,Material,Texto breve de material,Cantidad necesaria,Cantid.reducidas,Cantidad diferencia,Nombre del usuario,Fecha de necesidad,Orden,Nº reserva,Centro
35,01010-81230400361280,01010-81230,TORNILLO,6,NaN,3,U1205222,2024-04-01,400361280,2638118,3036


In [ ]:
df.to_excel("Prueba_15.xlsx", index = False)